# Projeto 1 - Ciência dos Dados

Nome: __Cauê Pileckas___

Nome: __Otávio Bianco___

Nome: __Victor Faria___

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
from unidecode import unidecode

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/cauedagostinho/Documents/Insper/2˚sem/Cdados/Projeto 1


Carregando a base de dados com os tweets classificados manualmente:

In [3]:
train = pd.read_excel('data/dados_treino.xlsx')
train

,Mensagem,Target
0,"Pelo menos no Livro 1 - A guerra dos tronos, a...",Conteúdo
1,Não gostei. O livro o tempo todo traz narrativ...,Conteúdo
2,"Ok, tem partes picantes legais, mas grande par...",Conteúdo
3,A estoria de amor entre duas pessoas psicologi...,Conteúdo
4,Péssimo livro. Ao mesmo tempo que tenta vanglo...,Conteúdo
...,...,...
495,"CLARO QUE AS ESTRELAS NÃO SÃO PAR A OBRA, MAS ...",Outros
496,Tenho muitos livros sobre a Segunda Guerra mun...,Conteúdo
497,Na época que estourou eu paguei 105 na edição ...,Conteúdo
498,Livro apenas comenta sobre o que é leitura ráp...,Outros


In [4]:
test = pd.read_excel('data/dados_teste.xlsx')
test

,Mensagem,Target
0,Um livro sensacional. Tenho aplicado o conceit...,Outros
1,A edição traz uma introdução que é puro spoile...,Conteúdo
2,É tão ruim quanto o canal dele.,Outros
3,É um livro baseado no fator motivacional e em ...,Conteúdo
4,"Olar, Frequentemente tendo sua marca, merecida...",Outros
...,...,...
245,Muito fraco e passional. Visão distorcida de d...,Outros
246,Nas primeiras páginas já temos o propósito do ...,Conteúdo
247,Achei um livro muito superficial e enfadonho. ...,Outros
248,"A menos que queira uma versão POCKET, não comp...",Outros


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

##### CONTEÚDO:

Relevante: Tweets que abordam o enredo, personagens, temas, qualidade da escrita, análises críticas, recomendações, citações do livro, discussões sobre a trama, etc.
Não relevante: Tweets que não se relacionam ao conteúdo do livro, como anúncios de venda, spam, tweets promocionais não relacionados ao conteúdo, mensagens pessoais não relacionadas ao livro, etc.

##### PRODUTO:

Relevante: Tweets que falam sobre a qualidade física do livro, como a capa, papel, encadernação, impressão, ilustrações, edições especiais, brindes, etc.
Não relevante: Tweets que não se referem à qualidade física do livro, como tweets promocionais da editora, anúncios de livros em geral, avaliações de transporte/demora de entrega, mensagens pessoais não relacionadas à qualidade física do livro, etc.

##### OUTROS: 

Relevante: Tweets que abordam temas gerais, não relacionados ao livro, mas que podem ser relevantes para os leitores, como dicas de leitura, eventos literários, escritores famosos, etc.
Não relevante: Tweets que não têm relação com o livro ou com interesses literários, como discussões sobre filmes, esportes, política, entre outros assuntos que não têm relevância para os critérios definidos.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    unidecode_text = unidecode(text_subbed)
    return unidecode_text.lower()

def retorna_lista_mensagens(df, target):
    lista_palavras_conteudo = []
    palavras_conteudo = []


    for index, row in df.iterrows():
        if row['Target'] == target:
            lista_palavras_conteudo.append(row['Mensagem'].split())
        
    for lista in lista_palavras_conteudo:
        for i in range(len(lista)):
            palavras_conteudo.append(lista[i])
            
    return palavras_conteudo

In [6]:
train['Mensagem'] = train['Mensagem'].apply(cleanup)

In [7]:
# lista_index_out = ['e', 'de', 'o', 'que', 'a', 'do', 'da', 'um', 'os', 'em', 'se', 'na', 'no']
lista_index_out = ["a", "o", "que", "do", "da", "de", "em", "um", "para", "e", "com", "nao", "uma", "os", "no", "se", "na", "por",
    "mais", "as", "dos", "como", "mas", "foi", "ao", "ele", "das", "tem", "à", "seu", "sua", "ou", "ser", "quando",
    "muito", "ha", "nos", "ja", "esta", "eu", "também", "so", "pelo", "pela", "ate", "isso", "ela", "entre", "era",
    "depois", "sem", "mesmo", "aos", "ter", "seus", "quem", "nas", "me", "esse", "eles", "estao", "voce", "tinha",
    "foram", "essa", "num", "nem", "suas", "meu", "as", "minha", "numa", "pelos", "elas", "havia", "seja",
    "qual", "sera", "nos", "tenho", "lhe", "deles", "essas", "esses", "pelas", "este", "fosse", "dele", "tu", "te",
    "voces", "vos", "lhes", "meus", "minhas", "teu", "tua", "teus", "tuas", "nosso", "nossa", "nossos", "nossas",
    "dela", "delas", "esta", "estes", "estas", "aquele", "aquela", "aqueles", "aquelas", "isto", "aquilo", "estou",
    "esta", "estamos", "estao", "estive", "esteve", "estivemos", "estiveram", "estava", "estavamos", "estavam",
    "estivera", "estiveramos", "esteja", "estejamos", "estejam", "estivesse", "estivessemos", "estivessem", "estiver",
    "estivermos", "estiverem", "hei", "ha", "havemos", "hao", "houve", "houvemos", "houveram", "houvera",
    "houveramos", "haja", "hajamos", "hajam", "houvesse", "houvessemos", "houvessem", "houver", "houvermos",
    "houverem", "houverei", "houverá", "houveremos", "houverão", "houveria", "houveríamos", "houveriam", "sou",
    "somos", "sao", "era", "eramos", "eram", "fui", "foi", "fomos", "foram", "fora", "foramos", "seja", "sejamos",
    "sejam", "fosse", "fossemos", "fossem", "for", "formos", "forem", "serei", "sera", "seremos", "serao", "seria",
    "seriamos", "seriam", "tenho", "temos", "tem", "tinha", "tinhamos", "tinham", "tive", "teve", "tivemos",
    "tiveram", "tivera", "tiveramos", "tenha", "tenhamos", "tenham", "tivesse", "tivessemos", "tivessem", "tiver",
    "tivermos", "tiverem", "terei", "tera", "teremos", "terao", "teria", "teriamos", "teriam"]

In [8]:
conteudo = retorna_lista_mensagens(train, 'Conteúdo')

# removendo palavras irrelevantes:
for palavra in conteudo:
    if palavra in lista_index_out:
        conteudo.remove(palavra)
    
serie_conteudo = pd.Series(conteudo)
df_conteudo_normalized = (serie_conteudo.value_counts(normalize=True)).to_frame()
df_conteudo_normalized = df_conteudo_normalized[~df_conteudo_normalized.index.isin(lista_index_out)]

conteudo_normalized = df_conteudo_normalized.copy()
conteudo_normalized['palavras'] = conteudo_normalized.index
conteudo_normalized = conteudo_normalized.reset_index(drop=True)
conteudo_normalized.head(10)

,0,palavras
0,0.032187,livro
1,0.007343,autor
2,0.006337,historia
3,0.004526,leitura
4,0.004426,bem
5,0.004325,sobre
6,0.004124,vida
7,0.003923,livros
8,0.003722,ler
9,0.003621,achei


In [9]:
produto = retorna_lista_mensagens(train, 'Produto')

# removendo palavras irrelevantes:
for palavra in produto:
    if palavra in lista_index_out:
        produto.remove(palavra)
        
serie_produto = pd.Series(produto)
df_produto_normalized = (serie_produto.value_counts(normalize=True)).to_frame()
df_produto_normalized = df_produto_normalized[~df_produto_normalized.index.isin(lista_index_out)]

produto_normalized = df_produto_normalized.copy()
produto_normalized['palavras'] = produto_normalized.index
produto_normalized = produto_normalized.reset_index(drop=True)
produto_normalized.head(10)

,0,palavras
0,0.029424,livro
1,0.008102,paginas
2,0.007676,livros
3,0.006823,bem
4,0.006397,versao
5,0.006397,edicao
6,0.005970,qualidade
7,0.005970,veio
8,0.005970,pra
9,0.004691,produto


In [10]:
outros = retorna_lista_mensagens(train, 'Outros')

# removendo palavras irrelevantes:
for palavra in outros:
    if palavra in lista_index_out:
        outros.remove(palavra)
        
serie_outros = pd.Series(outros)
df_outros_normalized = (serie_outros.value_counts(normalize=True)).to_frame()
df_outros_normalized = df_outros_normalized[~df_outros_normalized.index.isin(lista_index_out)]

outros_normalized = df_outros_normalized.copy()
outros_normalized['palavras'] = outros_normalized.index
outros_normalized = outros_normalized.reset_index(drop=True)
outros_normalized.head(10)

,0,palavras
0,0.029964,livro
1,0.007990,livros
2,0.006992,amazon
3,0.005993,produto
4,0.004794,preco
5,0.004794,ler
6,0.004594,leitura
7,0.004395,porem
8,0.003795,autor
9,0.003596,box


### Tratando o dataframe teste antes da análise

In [11]:
dataframe_teste = test
dataframe_teste['Mensagem'] = (test['Mensagem'].apply(cleanup)).to_frame()
dataframe_teste.head(5)

,Mensagem,Target
0,um livro sensacional tenho aplicado o conceito...,Outros
1,a edicao traz uma introducao que e puro spoile...,Conteúdo
2,e tao ruim quanto o canal dele,Outros
3,e um livro baseado no fator motivacional e em ...,Conteúdo
4,olar frequentemente tendo sua marca merecidame...,Outros


In [12]:
for index, row in dataframe_teste.iterrows():
    lista_feedback = (row['Mensagem'].split())
    
    for palavra in lista_feedback:
        if palavra in lista_index_out:
            lista_feedback.remove(palavra)
    
    feedback = " ".join(lista_feedback)
    
    row['Mensagem'] = feedback

In [13]:
data_teste = dataframe_teste.copy()

data_teste['Robot'] = None
data_teste

,Mensagem,Target,Robot
0,livro sensacional aplicado conceito adotar nov...,Outros,None
1,edicao traz introducao e puro spoiler muitos l...,Conteúdo,None
2,tao ruim quanto canal,Outros,None
3,livro baseado fator motivacional apanhado tecn...,Conteúdo,None
4,olar frequentemente tendo marca merecidamente ...,Outros,None
...,...,...,...
245,fraco passional visao distorcida direita leria...,Outros,None
246,primeiras paginas temos proposito livro pura r...,Conteúdo,None
247,achei livro superficial enfadonho autor fica d...,Outros,None
248,menos queira versao pocket compre descricao pr...,Outros,None


In [14]:
# testando o método de análise
# line_conteudo = conteudo_normalized.loc[conteudo_normalized['palavras'] == 'e']
# probabilidade_conteudo = (line_conteudo[0].values)[0]
# probabilidade_conteudo

### Suavização de Laplace

In [15]:
df_teste = data_teste.copy()

prod_probs_conteudo = 1
prod_probs_produto = 1
prod_probs_outros = 1

for index, row in df_teste.iterrows():
    lista_dividida = row['Mensagem'].split()
    suavizador = False
    #print(lista_dividida)
    
    # conteúdo
    for palavra in lista_dividida:
        #print(palavra)
        line_conteudo = conteudo_normalized.loc[conteudo_normalized['palavras'] == palavra]
        if len(line_conteudo)==1:
            probabilidade_conteudo = line_conteudo.values[0][0]          #(line_conteudo[0].values)[0] -- anterior -- 
            prod_probs_conteudo *= probabilidade_conteudo
        else:
            suavizador=True 
     #produto   
    for palavra in lista_dividida:
        line_produto = produto_normalized.loc[produto_normalized['palavras'] == palavra]
        if len(line_produto)==1:
            probabilidade_produto = line_produto.values[0][0] 
            prod_probs_produto *= probabilidade_produto
        else:
            suavizador= True
     
    #outros   
    for palavra in lista_dividida:
        line_outros = outros_normalized.loc[outros_normalized['palavras'] == palavra]
        if len(line_outros)==1:
            probabilidade_outros = line_outros.values[0][0] 
            prod_probs_outros *= probabilidade_outros
        else:
            suavizador=True
    
    if suavizador==False:
        if prod_probs_conteudo == max([prod_probs_conteudo, prod_probs_produto, prod_probs_outros]):
            row['Robot'] = 'Conteúdo'
        
        elif prod_probs_produto == max([prod_probs_conteudo, prod_probs_produto, prod_probs_outros]):
            row['Robot'] = 'Produto'
        
        elif prod_probs_outros == max([prod_probs_conteudo, prod_probs_produto, prod_probs_outros]):
            row['Robot'] = 'Outros'
    else:
        x=len(lista_dividida)
        if prod_probs_conteudo/x == max([(prod_probs_conteudo/x), (prod_probs_produto/x), (prod_probs_outros/x)]):
            row['Robot'] = 'Conteúdo'
        
        elif prod_probs_produto/x == max([(prod_probs_conteudo/x), (prod_probs_produto/x), (prod_probs_outros/x)]):
            row['Robot'] = 'Produto'
        
        elif prod_probs_outros/x == max([(prod_probs_conteudo/x), (prod_probs_produto/x), (prod_probs_outros/x)]):
            row['Robot'] = 'Outros'
df_teste

,Mensagem,Target,Robot
0,livro sensacional aplicado conceito adotar nov...,Outros,Produto
1,edicao traz introducao e puro spoiler muitos l...,Conteúdo,Produto
2,tao ruim quanto canal,Outros,Produto
3,livro baseado fator motivacional apanhado tecn...,Conteúdo,Produto
4,olar frequentemente tendo marca merecidamente ...,Outros,Produto
...,...,...,...
245,fraco passional visao distorcida direita leria...,Outros,Conteúdo
246,primeiras paginas temos proposito livro pura r...,Conteúdo,Conteúdo
247,achei livro superficial enfadonho autor fica d...,Outros,Conteúdo
248,menos queira versao pocket compre descricao pr...,Outros,Conteúdo


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

#### Verificando inicialmente com "crosstab"

In [16]:
#tabela = pd.crosstab(df_teste['Target'], df_teste['Robot'])
tabela = pd.crosstab(df_teste['Robot'], df_teste['Target'], normalize=True, margins=True)
tabela = tabela*100
tabela

Target,Conteúdo,Outros,Produto,All
Robot,,,,
Conteúdo,52.0,31.6,14.4,98.0
Produto,0.8,1.2,0.0,2.0
All,52.8,32.8,14.4,100.0


#### Encontrando percentuais de Falsos Targets e Verdadeiros Targets

In [17]:
percent_verdadeiros_cont = len(df_teste.loc[(df_teste['Robot'] == 'Conteúdo') & (df_teste['Target'] == 'Conteúdo')]) / len(df_teste)
percent_falsos_cont = len(df_teste.loc[(df_teste['Robot'] == 'Conteúdo') & (df_teste['Target'] != 'Conteúdo')]) / len(df_teste)


percent_verdadeiros_prod = len(df_teste.loc[(df_teste['Robot'] == 'Produto') & (df_teste['Target'] == 'Produto')]) / len(df_teste)
percent_falsos_prod = len(df_teste.loc[(df_teste['Robot'] == 'Produto') & (df_teste['Target'] != 'Produto')]) / len(df_teste)

percent_verdadeiros_outros = len(df_teste.loc[(df_teste['Robot'] == 'Outros') & (df_teste['Target'] == 'Outros')]) / len(df_teste)
percent_falsos_outros = len(df_teste.loc[(df_teste['Robot'] == 'Outros') & (df_teste['Target'] != 'Outros')]) / len(df_teste)

print(f'Verdadeiros "Conteúdo": {percent_verdadeiros_cont*100:.2f}% | Falsos "Conteúdo": {percent_falsos_cont*100:.2f}%')
print(f'Verdadeiros "Produto": {percent_verdadeiros_prod*100:.2f}%   | Falsos "Produto": {percent_falsos_prod*100:.2f}%')
print(f'Verdadeiros "Outros": {percent_verdadeiros_outros*100:.2f}%    | Falsos "Outros": {percent_falsos_outros*100:.2f}%')

Verdadeiros "Conteúdo": 52.00% | Falsos "Conteúdo": 46.00%
Verdadeiros "Produto": 0.00%   | Falsos "Produto": 2.00%
Verdadeiros "Outros": 0.00%    | Falsos "Outros": 0.00%


#### Encontrando Acurácia do robô

In [18]:
df_acuracia = df_teste.copy()

In [19]:
df_acuracia.loc[df_acuracia['Target'] == df_acuracia['Robot'], 'Acuracia'] = 1
df_acuracia['Acuracia_sum'] = df_acuracia['Acuracia'].sum()

acuracia = (df_acuracia['Acuracia_sum']).values[-1]
acuracia_percent = (acuracia / len(df_acuracia)) * 100

print(f'Acurácia do robô: {acuracia} mensagens, equivalente a {acuracia_percent}% das mensagens')

Acurácia do robô: 130.0 mensagens, equivalente a 52.0% das mensagens


___
### Concluindo

#### Respondendo questões:

1: Faça um comparativo qualitativo sobre os percentuais obtidos para que possa discutir a performance do seu classificador.

Resposta: O classificador teve uma melhor performance quando se trata de classificar as mensagens como "Conteúdos", isso se da pois no banco de dados "treino" a probabilidade da junção das palavras que formam alguma frase foi maior sempre que classificado como "Conteúdos".

2: 

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

In [21]:
df_treino = train

for index, row in df_treino.iterrows():
    lista_feedback = (row['Mensagem'].split())
    
    for palavra in lista_feedback:
        if palavra in lista_index_out:
            lista_feedback.remove(palavra)
    
    feedback = " ".join(lista_feedback)
    
    row['Mensagem'] = feedback

In [36]:
df_total = pd.concat([dataframe_teste, df_treino], ignore_index=True)

repeticoes = 100

for i in range(repeticoes):
    # Embaralhando as linhas do DataFrame
    df_embaralhado = df_total.sample(frac=1).reset_index(drop=True)
    
    # Divida o DataFrame em duas partes: 500 e 250 linhas
    parte1 = df_embaralhado[:500]
    parte2 = df_embaralhado[500:750]



,Mensagem,Target
0,comprei imaginando guia moderno investir contr...,Conteúdo
1,autor fala fala diz nada diz se trata nada ale...,Conteúdo
2,amor deus preco esteso brasil porque livro ain...,Conteúdo
3,aterei historia sucesso consagrado edicao bols...,Outros
4,traducao dessa edicao chega pes disponivel com...,Produto
...,...,...
495,pior livro li toda minha vida enrola fala livr...,Conteúdo
496,possuo versao impressa livro guerra tronos ont...,Produto
497,trilogia pior coisa li junto crepusculo coisa ...,Conteúdo
498,baixei livro motivada alto indice avaliacoes p...,Conteúdo


___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**